In [ ]:
import os
from os.path import join
import argparse
import numpy as np
from functools import reduce
import glob

import torch
from pyquaternion import Quaternion
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import transform_matrix

In [ ]:
DIR_DATA = "/ssd/Datasets_and_code/nuscenes_depth_estimation/dataset/dataset_radar_cam"
DIR_NUSCENES = "/ssd/Datasets_and_code/nuscenes_depth_estimation/dataset/nuscenes_mini"
VERSION = "v1.0-mini"

In [ ]:
def get_intrinsic_matrix(nusc, cam_token):       
    cam_data = nusc.get('sample_data', cam_token)
    cs_rec = nusc.get('calibrated_sensor', cam_data['calibrated_sensor_token'])
    
    return np.array( cs_rec['camera_intrinsic'] )

In [ ]:
nusc = NuScenes(VERSION, dataroot = DIR_NUSCENES, verbose=False)
dir_data_out = join(DIR_DATA, 'cam_matrix')
if not os.path.exists(dir_data_out):
    os.makedirs(dir_data_out)

# Remove all files in the output folder
f_list=glob.glob(join(dir_data_out,'*'))
for f in f_list:
    os.remove(f)
print('removed %d old files in output folder' % len(f_list))

In [ ]:
sample_indices = torch.load(join(DIR_DATA,'data_split.tar'))['all_indices'] 

for sample_idx in sample_indices:

    cam_token = nusc.sample[sample_idx]['data']['CAM_FRONT']
    cam_data = nusc.get('sample_data', cam_token) 

    K = get_intrinsic_matrix(nusc, cam_token)

    np.savez(join(dir_data_out, '%05d_cam_matrix.npz' % sample_idx), K=K)    